<a href="https://colab.research.google.com/github/yuuki-kusumoto/kusumoto/blob/main/ksumoto_BERT_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install MeCab
!pip install mecab-python3
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

import numpy as np

!pip install ipadic
!pip install mecab-python3
!pip install unidic-lite
# MeCabとtransformersを用意する
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# 以下で報告があるようにmecab-python3のバージョンを0.996.5にしないとtokezerで落ちる
# https://stackoverflow.com/questions/62860717/huggingface-for-japanese-tokenizer
!pip install mecab-python3==0.996.5
!pip install unidic-lite # これないとMeCab実行時にエラーで落ちる
!pip install transformers
!pip install "transformers==2.5.1"
!pip install "torchtext== 0.11"

import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

import torch
import transformers
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

# パッケージのimport
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import torchtext

# 必要なパッケージのimport
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext


! curl http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/pn.csv.m3.120408.trim > pn.csv


model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# model_nameはここから取得(cf. https://huggingface.co/transformers/pretrained_models.html)
model_name = "cl-tohoku/bert-base-japanese"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

import warnings
warnings.filterwarnings('ignore')

BERTのソースコード

In [ ]:
from torch import nn

class BertForReview(nn.Module):
    '''BERTモデルにレビュー文の2クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForReview, self).__init__()

        # BERTモジュール
        #self.bert = model  # 日本語学習済みのBERTモデル

        self.bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking',
                                              output_attentions=True,
                                              output_hidden_states=True)

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は14クラス
        #cls層の追加
        self.cls = nn.Linear(in_features=768, out_features=2)
        
        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)    
    
    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        #print("input_ids :", input_ids)
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output
        #result, _, attentions = self.bert(input_ids, output_attentions=True)

        all_attentions = result[2]
        last_layer_attention = all_attentions[-1]
        hidden_states = result[3]

        m = nn.Softmax(dim=1)

        #vec1 = self._get_cls_vec(hidden_states[-1])
        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_02 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素。　つまりclsトークンを獲得している。
        vec_03 = vec_02.view(-1, 768)  # sizeを[batch_size, hidden_size] clsトークン
        #vec_03 = vec_0[:, 0:100, :] #文章の埋め込み表現
        output = self.cls(vec_03)  # 全結合層
        A = F.log_softmax(output, dim=1)
        B = m(output)

        #return vec_02, vec_03, last_layer_attention,hidden_states  #vec_02 : clsトークン, #vec_03 : 文章の特徴量
        
        return m(output), all_attentions, vec_02#, vec_03


# モデル構築
BERT_net = BertForReview()

# 訓練モードに設定
BERT_net.train()

print('ネットワーク設定完了')


# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in BERT_net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in BERT_net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in BERT_net.cls.parameters():
    param.requires_grad = True

# BERTの元の部分はファインチューニング
bert_optimizer = optim.Adam([
    {'params': BERT_net.bert.encoder.layer[-1].parameters(), 'lr': 1e-4},
    {'params': BERT_net.cls.parameters(), 'lr': 1e-3}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()


Transformer Encoder のソースコード

In [ ]:
from torch import nn
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext

a_po = "朝食ポジティブ"
a_po = tokenizer.encode(a_po)
#print(a_po)
a_po_to = torch.LongTensor(a_po).unsqueeze(0)
#print(a_po_to)
#clsトークン
ou = model(a_po_to)
ou_0 = ou[0]
ou_1 = ou_0[:, 0, :]
ou_2 = ou_1.view(-1, 768)
ou_3 = ou_2[0]


a_ne = "朝食ネガティブ"
a_ne = tokenizer.encode(a_ne)
a_ne_to = torch.LongTensor(a_ne).unsqueeze(0)
#clsトークン
ou2 = model(a_ne_to)
ou2_0 = ou2[0]
ou2_1 = ou2_0[:, 0, :]
ou2_2 = ou2_1.view(-1, 768)
ou2_3 = ou2_2[0]

ou_4 = ou_3.tolist()
ou2_4 = ou2_3.tolist()

#TransformerClasification層でのclsトークンの変換
def LabelCls_convert(CLS):
  
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  m = nn.Softmax(dim=1)
  CLS2 = m(CLS)

  _, PRED = torch.max(CLS, 1)  # ラベルを予測
  a = CLS.dtype
  cls_list = []
  for i in range(len(PRED)):
    if PRED[i] == 1:
      cls_list.append(ou_4)
    else:
      cls_list.append(ou2_4)
  cls_numpy = np.array(cls_list)
  labelcls = torch.tensor(cls_numpy)
  labelcls = labelcls.to(device)
  labelcls = labelcls.to(a)

  return labelcls, PRED, CLS

class PositionalEncoder(nn.Module):
    '''入力された単語の位置を示すベクトル情報を付加する'''

#max_seq_len はパディングしないといけない？
    def __init__(self, d_model=768, max_seq_len=100):
        super().__init__()

        self.d_model = d_model  # 単語ベクトルの次元数

        # 単語の順番（pos）と埋め込みベクトルの次元の位置（i）によって一意に定まる値の表をpeとして作成
        pe = torch.zeros(max_seq_len, d_model)
        #print(pe)

        # GPUが使える場合はGPUへ送る
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        pe = pe.to(device)

        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos /
                                          (10000 ** ((2 * (i + 1))/d_model)))

        # 表peの先頭に、ミニバッチ次元となる次元を足す
        self.pe = pe.unsqueeze(0)

        # 勾配を計算しないようにする
        self.pe.requires_grad = False

    def forward(self, x):

        # 入力xとPositonal Encodingを足し算する
        # xがpeよりも小さいので、大きくする
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret

class Attention(nn.Module):
    '''シングルAttentionで実装'''

    def __init__(self, d_model=768):
        super().__init__()

        # SAGANでは1dConvを使用したが、今回は全結合層で特徴量を変換する
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)

        # 出力時に使用する全結合層
        self.out = nn.Linear(d_model, d_model)

        # Attentionの大きさ調整の変数
        self.d_k = d_model

    def forward(self, pe):
      #mask = torch.zeros(768)
      #for i in range(0, len(cls_tokens)):
        
        # 全結合層で特徴量を変換
        k = self.k_linear(pe)
        q = self.q_linear(pe)
        v = self.v_linear(pe)

        print(k)
        print("AAAAAAAAAAA", k.shape)

        # Attentionの値を計算する
        # 各値を足し算すると大きくなりすぎるので、root(d_k)で割って調整
        
        #attention_weights = torch.matmul(q, k.transpose(2, 1)) / math.sqrt(self.d_k)

        #attention_weights = torch.matmul(q.unsqueeze(1), k.transpose(2, 1) )/ math.sqrt(self.d_k)
        attention_weights = torch.matmul(q, k.transpose(2, 1) )/ math.sqrt(self.d_k)


        # ここでmaskを計算
        #mask = mask.unsqueeze(1)
        #attention_weights = attention_weights.masked_fill(mask == 0, -1e9)

        # softmaxで規格化をする
        normlized_weights = F.softmax(attention_weights, dim=-1)

        # AttentionをValueとかけ算
        output = torch.matmul(normlized_weights, v)

        # 全結合層で特徴量を変換
        output = self.out(output)
        
        return output, normlized_weights

class MultiHeadAttention(nn.Module):

  def __init__(self, d_model, classes, num_heads):
    super().__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    self.head_dim = d_model // num_heads
    self.seq_length = classes
   
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = nn.Linear(d_model, d_model)
    self.wk = nn.Linear(d_model, d_model)
    self.wv = nn.Linear(d_model, d_model)

    self.d_k = d_model

    # 出力時に使用する全結合層
    self.out = nn.Linear(d_model, d_model)
        # 出力時に使用する全結合層
    self.out = nn.Linear(d_model, d_model)
    self._reset_parameters()
        
 ## def split_heads(self, x, batch_size):
    """最後の次元を(num_heads, depth)に分割。
    結果をshapeが(batch_size, num_heads, seq_len, depth)となるようにリシェイプする。
    """
  ##  x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
 ##   return torch.transpose(x, perm=[0, 2, 1, 3])

  def _reset_parameters(self):
    # Original Transformer initialization, see PyTorch documentation
    nn.init.xavier_uniform_(self.wq.weight)
    self.wq.bias.data.fill_(0)

    nn.init.xavier_uniform_(self.wk.weight)
    self.wk.bias.data.fill_(0)

    nn.init.xavier_uniform_(self.wv.weight)
    self.wv.bias.data.fill_(0)

    
  def forward(self, pe):
    
    #batch_sizeを指定
    batch_size = 4
    
    q = self.wq(pe)
    k = self.wk(pe)
    v = self.wv(pe)
    d_k = self.d_k


    q = q.reshape(batch_size, self.seq_length, self.num_heads, self.head_dim)
    k = k.reshape(batch_size, self.seq_length, self.num_heads, self.head_dim)
    v = v.reshape(batch_size, self.seq_length, self.num_heads, self.head_dim)


    #q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    #k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    #v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    
    #attention_weights = torch.matmul(q, k.transpose(-2, -1))
    #attention_weights = attention_weights / math.sqrt(d_k)

    #attention_weights = torch.matmul(q.unsqueeze(1), k.transpose(-2, -1) )/ math.sqrt(self.d_k)
    attention_weights = torch.matmul(q, k.transpose(3, 2) )/ math.sqrt(self.d_k)  
    
    # ここでmaskを計算
    #mask = mask.unsqueeze(1)
    #attention_weights = attention_weights.masked_fill(mask == 0, -1e9)
    # softmaxで規格化をする
    
    normlized_weights = F.softmax(attention_weights, dim=-1)
    # AttentionをValueとかけ算
    values = torch.matmul(normlized_weights, v)
    # 全結合層で特徴量を変換

    values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
    values = values.reshape(batch_size,self.seq_length, self.d_model)
    output = self.out(values)
    
    return output, normlized_weights


class FeedForward(nn.Module):
    def __init__(self, d_model = 768, d_ff=1536, dropout=0.1):
        '''Attention層から出力を単純に全結合層2つで特徴量を変換するだけのユニットです'''
        super().__init__()

        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, d_model=768, dropout=0.1):
        super().__init__()

        # LayerNormalization層
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)

        # Attention層
        #self.attn = Attention(d_model=d_model)
        self.attn = MultiHeadAttention(d_model=768, classes=14,  num_heads=4)

        # Attentionのあとの全結合層2つ
        self.ff = FeedForward(d_model)

        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self,  x):
        # PositionalEncoding を行った後に正規化する
        x_normlized = self.norm_1(x)

        output, normlized_weights = self.attn(x_normlized)
        #FeedForward層の入力作成
        x2 = x + self.dropout_1(output)

        # 正規化と全結合層
        x_normlized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normlized2))


        return output, normlized_weights

#MLP層
class ClassificationHead(nn.Module):
    '''Transformer_Blockの出力を使用し、最後にクラス分類させる'''

    def __init__(self, d_model=768, d_model2=384, output_dim=14):
        super().__init__()

        # 全結合層
        self.linear1 = nn.Linear(d_model, d_model2)  # output_dimはポジ・ネガの2つ
        self.linear2 = nn.Linear(d_model2, output_dim)  # output_dimはポジ・ネガの2つ

        # 重み初期化処理
        nn.init.normal_(self.linear1.weight, std=0.02)
        nn.init.normal_(self.linear1.bias, 0)
        # 重み初期化処理
        nn.init.normal_(self.linear2.weight, std=0.02)
        nn.init.normal_(self.linear2.bias, 0)

    def forward(self, x):
      #バッチサイズに合わせて変化
        #x0 = torch.stack([torch.mean(x[0,:,:],0), torch.mean(x[1,:,:], 0), torch.mean(x[2,:,:],0), torch.mean(x[3,:,:], 0)], 0)
        x0 = x[:, 0, :]  # 各ミニバッチの各文の cls の特徴量（768次元）を取り出す
        x1 = F.relu(self.linear1(x0))
        x2 = self.linear2(x1)

        return x0, x2
        
# 最終的なTransformerモデルのクラス

class TransformerClassification(nn.Module):
    '''Transformerでクラス分類させる'''

    def __init__(self,  d_model=768, max_seq_len=14, d_model2=384, output_dim=14):
        super().__init__()

        # モデル構築
        
        #self.net1 = BERT_net
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim,d_model2=d_model2,  d_model=d_model)

    def forward(self, bert_cls1, x1, bert_cls2, x2, bert_cls3, x3, bert_cls4, x4, bert_cls5, x5, bert_cls6, x6, bert_cls7, x7):

        q = torch.stack([ bert_cls1, x1, bert_cls2, x2, bert_cls3, x3, bert_cls4, x4, bert_cls5, x5, bert_cls6, x6, bert_cls7, x7], dim=1)
        
        q2 = self.net2(q)

        x13, normlized_weights_1 = self.net3_1(q2)  # Self-Attentionで特徴量を変換
        #x3_1_2 = x13[:, 0, :]

        x3_2, normlized_weights_2 = self.net3_2(x13)  # Self-Attentionで特徴量を変換        
       
        tr_cls1, tr_cls2 = self.net4(x3_2)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力        [Transformer層での予測]


        OO = nn.Sigmoid()
        return OO(tr_cls2)
      
        #return OO(total_cls)  #normlized_weights_1, #normlized_weights_2


読み込んだテキストをトークナイズして, バッチサイズ 4 に分割します. BERT モデルを 14 個読み込んで transformer を接続するアンサンブル学習モデルの場合, バッチサイズ 4 を超えるとメモリがクラッシュしてしまうと思います. 単一の BERT モデルであればバッチサイズは何でもいいと思います.

In [ ]:
# 前から2割をテストデータとする
tr_df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(tr_df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
tr_df[len_0_2:].to_csv("./train.tsv", sep='\t', index=False, header=None)
print(tr_df[len_0_2:].shape)

def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=100, return_tensors='pt')[0]


TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,include_lengths=True, batch_first=True, fix_length=100, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL1 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL2 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL3 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL4 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL5 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL6 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL7 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL8 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL9 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL10 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL11 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL12 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL13 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

#ラベルは数字なのでラベル用オブジェクトはsequential = False
LABEL14 = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1938  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：4000個、test：2000個

#dataset_train, dataset_eval = dataset_train_eval.split(split_ratio = 1 - 0.2, random_state=random.seed(1234))

# 各tsvファイルを読み込み、分かち書きをしてdatasetに
# train_eval：61300個、test：15324個
dataset_train_eval, dataset_test = torchtext.legacy.data.TabularDataset.splits(
    path='.', train='train.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label1', LABEL1), ('Label2', LABEL2), ('Label3', LABEL3),  ('Label4', LABEL4), ('Label5', LABEL5), ('Label6', LABEL6), ('Label7', LABEL7), ('Label8', LABEL8), ('Label9', LABEL9), ('Label10', LABEL10), ('Label11', LABEL11), ('Label12', LABEL12), ('Label13', LABEL13), ('Label14', LABEL14)])
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：4000個、test：2000個

dataset_train, dataset_eval = dataset_train_eval.split(split_ratio = 0.3, random_state=random.seed(1234))

# datasetの長さを確認してみる
print("train", dataset_train.__len__())
print("eval", dataset_eval.__len__())
print("test", dataset_test.__len__())

# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  # 長さを確認 [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベル：", item.Label1)

# DataLoaderを作成

batch_size = 4

dl_train = torchtext.legacy.data.Iterator(
    dataset_train_eval, batch_size=batch_size, train=True, shuffle=False)

dl_eval = torchtext.legacy.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.legacy.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False, shuffle=False)

print(len(dl_train))
print(len(dl_eval))
print(len(dl_test))

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_test}

BERT の訓練用の関数

In [ ]:
# モデルを学習させる関数を作成
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  
print("使用デバイス：", device)
print('-----start-------')

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    

    # ネットワークをGPUへ
    net.to(device)

    train_loss1 = []
    val_loss1 = []
    F1_list = []
    F1_list2 = []
    
    precision_list1 = []
    precision_list2 = []
    
    recall_list1 = []
    recall_list2 = []

    acc_train_list1 = []
    acc_val_list1 = []

    acc_train_list1 = []
    acc_val_list1 = []

    train_loss2 = []
    val_loss2 = []

    acc_train_list2 = []
    acc_val_list2 = []

    acc_train_list2 = []
    acc_val_list2 = []

    total_label_list = []
    out_list = []

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        #for phase in ['train', 'val']:
            
            phase="train"            

            pred_list = []
            label_list = []

            net.train()  # モデルを訓練モードに
            
            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict["train"]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label1.to(device)  # ラベル
                labels2 = batch.Label2.to(device)
                labels3 = batch.Label3.to(device)
                labels4 = batch.Label4.to(device)
                labels5 = batch.Label5.to(device)
                labels6 = batch.Label6.to(device)
                labels7 = batch.Label7.to(device)
                labels8 = batch.Label8.to(device)
                labels9 = batch.Label9.to(device)
                labels10 = batch.Label10.to(device)
                labels11 = batch.Label11.to(device)
                labels12 = batch.Label12.to(device)
                labels13 = batch.Label13.to(device)
                labels14 = batch.Label14.to(device)
                pa = torch.stack([labels, labels2, labels3, labels4, labels5,labels6,labels7,labels8,labels9,labels10, labels11, labels12,labels13,labels14], dim = 1)
                pa = torch.tensor(pa,  dtype=torch.float32)
                

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                #with torch.set_grad_enabled(phase == 'train'):    

                tr_cls = net(inputs)

#Transformer層の出力での予測
                loss = criterion(tr_cls, pa)


                #LABEL_cls, BERT_pred = LabelCls_convert(outputs)
                ang = torch.round(tr_cls)
                aa = ang.cpu().detach().numpy()
                bb = pa.cpu().detach().numpy()
                pred_list = np.append(pred_list, aa)
                label_list = np.append(label_list, bb)


#Transformer層の出力での予測

                #if phase == "train":
                # 訓練時は逆誤差伝搬
                loss.backward()
                optimizer.step()
                        
                # 損失と正解数の合計を更新
                epoch_loss += loss.item() * batch_size


            # epochごとのlossと正解率
#BERT       
            #f12 = sum(f1) / len(f1)
            #F1_list = np.append(F1_list, f12)
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            precisionscore = precision_score(pred_list.reshape(int(len(pred_list)/14), 14), label_list.reshape(int(len(label_list)/14), 14), average= 'micro') 
            recallscore = recall_score(pred_list.reshape(int(len(pred_list)/14), 14), label_list.reshape(int(len(label_list)/14), 14), average= 'micro')
            f1score = f1_score(pred_list.reshape(int(len(pred_list)/14), 14), label_list.reshape(int(len(label_list)/14), 14), average= 'micro')
            

#Transformer


            #if phase == 'train':
            train_loss1 = np.append(train_loss1, epoch_loss)
            print("f1",f1score)
            print("precision", precisionscore)
            print("recaoll", recallscore)
            precision_list1 = np.append(precision_list1, precisionscore)
            recall_list1 = np.append(recall_list1, recallscore)
            F1_list2 = np.append(F1_list2, f1score)

              #cm1 = confusion_matrix(label_list, pred_list)                       #Transformer
                  #BERT
              
              #print("\n BERT 混合行列", cm1, sep="\n")

            print('Epoch of BERT {}/{} | {:^5} | Loss: {:.4f} F1: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss, f1score))
            
    
    #out_list = out_list.reshape(int(len(out_list)/2), 2)
    return net,  train_loss1, val_loss1, F1_list2, F1_list, precision_list1, precision_list2, recall_list1, recall_list2, pred_list

BERT の訓練用のコード

In [ ]:
f12=0
ec=0
for i in range(1,11, 1):

  # モデル構築
  BERT_net = BertForReview()
  # 訓練モードに設定
  BERT_net.train()
  # 1. まず全部を、勾配計算Falseにしてしまう
  for param in BERT_net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
  for param in BERT_net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
  for param in BERT_net.cls.parameters():
    param.requires_grad = True
  
  B = i/10000000

  bert_optimizer = optim.Adam([
    {'params': BERT_net.bert.encoder.layer[-1].parameters(), 'lr1': 6e-6},
    {'params': BERT_net.cls.parameters(), 'lr2': B}
  ])
  print("lr", B)

  net_trained, trainloss, valloss, Ftrain, Fval, Pretrain1, Preval1, Rectrain1, Recval1,  PRED_LIST1 = train_model(BERT_net,  dataloaders_dict,criterion, bert_optimizer, num_epochs=20)

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  net_trained.eval()   # モデルを検証モードに
  net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
  epoch_corrects = 0
  pred_list1 = []
  label_list1 = []
  aa1 = 0
  bb1 = 0

  acc_count=0
  part_acc_count=0
  all_miss=0


  for batch in tqdm(dl_test): 
    inputs = batch.Text[0].to(device)  # 文章
    labels0 = batch.Label1.to(device)  # ラベル
    labels1 = batch.Label2.to(device)
    labels2 = batch.Label3.to(device)
    labels3 = batch.Label4.to(device)
    labels4 = batch.Label5.to(device)
    labels5 = batch.Label6.to(device)
    labels6 = batch.Label7.to(device)
    labels7 = batch.Label8.to(device)
    labels8 = batch.Label9.to(device)
    labels9 = batch.Label10.to(device)
    labels10 = batch.Label11.to(device)
    labels11 = batch.Label12.to(device)
    labels12 = batch.Label13.to(device)
    labels13 = batch.Label14.to(device)
    pa = torch.stack([labels0, labels1, labels2, labels3, labels4,labels5,labels6,labels7,labels8,labels9, labels10, labels11,labels12,labels13], dim = 1)
    pa2 = pa.cpu().numpy()
      

    Outputs = net_trained(inputs)

    preds = torch.round(Outputs)

    aa1 = preds.cpu().detach().numpy()
    bb1 = pa.cpu().detach().numpy()
    pred_list1 = np.append(pred_list1, aa1)
    label_list1 = np.append(label_list1, bb1)

  precisionscore = precision_score(pred_list1.reshape(int(len(pred_list1)/14), 14), label_list1.reshape(int(len(label_list1)/14), 14), average= 'micro') 
  recallscore = recall_score(pred_list1.reshape(int(len(pred_list1)/14), 14), label_list1.reshape(int(len(label_list1)/14), 14), average= 'micro')
  epoch_f1 = f1_score(pred_list1.reshape(int(len(pred_list1)/14), 14), label_list1.reshape(int(len(label_list1)/14), 14), average= 'micro')
        
  print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), precisionscore))
        
  print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), recallscore))
        
  print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_f1))

  if epoch_f1>f12:
    f12=epoch_f1
    ec=ec+1

  if ec>6:
    print("########################################################################################################################################################")

  PRED_LIST = pred_list1.reshape(int(len(pred_list1)/14), 14)
  LABEL_LIST = label_list1.reshape(int(len(label_list1)/14), 14)

  for i in range(len(PRED_LIST)):
    if (PRED_LIST[i]==LABEL_LIST[i]).all():
      acc_count+=1
    elif (PRED_LIST[i]==LABEL_LIST[i]).any():
      part_acc_count+=1
  all_miss = len(dl_test.dataset) - (acc_count + part_acc_count)
  
  print("######################################################")
  print("ACC_COUNT", acc_count)
  print("PART_ACC_COUNT", part_acc_count)
  print("ALL_MISS", all_miss)

作成した事前学習済みの BERT ベースの 2 値分類モデルを net_trained11 などに格納して, それぞれのモデルから CLS トークンを抽出する関数

In [ ]:
  # GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)
print('-----start-------')

def train_model(net_trained11, net_trained12, net_trained21, net_trained22, net_trained31, net_trained32, net_trained41, net_trained42, net_trained51, net_trained52, net_trained61, net_trained62, net_trained71, net_trained72, dataloaders_dict, criterion, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # ネットワークをGPUへ
    
    net_trained11.eval()   # モデルを検証モードに
    net_trained11.to(device)  # GPUが使えるならGPUへ送る

    net_trained12.eval()   # モデルを検証モードに
    net_trained12.to(device)  # GPUが使えるならGPUへ送る

    
    net_trained21.eval()   # モデルを検証モードに
    net_trained21.to(device)  # GPUが使えるならGPUへ送る

    net_trained22.eval()   # モデルを検証モードに
    net_trained22.to(device)  # GPUが使えるならGPUへ送る

    
    net_trained31.eval()   # モデルを検証モードに
    net_trained31.to(device)  # GPUが使えるならGPUへ送る

    net_trained32.eval()   # モデルを検証モードに
    net_trained32.to(device)  # GPUが使えるならGPUへ送る

    
    net_trained41.eval()   # モデルを検証モードに
    net_trained41.to(device)  # GPUが使えるならGPUへ送る

    net_trained42.eval()   # モデルを検証モードに
    net_trained42.to(device)  # GPUが使えるならGPUへ送る

    
    net_trained51.eval()   # モデルを検証モードに
    net_trained51.to(device)  # GPUが使えるならGPUへ送る

    net_trained52.eval()   # モデルを検証モードに
    net_trained52.to(device)  # GPUが使えるならGPUへ送る

    
    net_trained61.eval()   # モデルを検証モードに
    net_trained61.to(device)  # GPUが使えるならGPUへ送る

    net_trained62.eval()   # モデルを検証モードに
    net_trained62.to(device)  # GPUが使えるならGPUへ送る

    
    net_trained71.eval()   # モデルを検証モードに
    net_trained71.to(device)  # GPUが使えるならGPUへ送る

    net_trained72.eval()   # モデルを検証モードに
    net_trained72.to(device)  # GPUが使えるならGPUへ送る


    train_loss1 = []
    val_loss1 = []
    F1_list = []
    F1_list2 = []

    acc_train_list1 = []
    acc_val_list1 = []

    acc_train_list1 = []
    acc_val_list1 = []

    train_loss2 = []
    val_loss2 = []

    acc_train_list2 = []
    acc_val_list2 = []

    acc_train_list2 = []
    acc_val_list2 = []

    total_label_list = []
    out_list = []

    INP=[]



    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        ext=[]
        b1=[]
        b2=[]
        b3=[]
        b4=[]
        b5=[]
        b6=[]
        b7=[]

        c1=[]
        c2=[]
        c3=[]
        c4=[]
        c5=[]
        c6=[]
        c7=[]

        p = []
        ins=[]



        f1 = 0
        f12 = 0
        f1_list = []
        f1_list2 = []

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            
            pred_list = []
            label_list = []

            pred_list2 = []
            label_list2 = []

            epoch_loss = 0.0  # epochの損失和
            epoch_loss1 = 0.0  # epochの損失和
            epoch_corrects1 = 0  # epochの正解数
            epoch_loss2 = 0.0  # epochの損失和
            epoch_corrects2 = 0  # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書オブジェクト

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels0 = batch.Label1.to(device)  # ラベル
                labels1 = batch.Label2.to(device)
                labels2 = batch.Label3.to(device)
                labels3 = batch.Label4.to(device)
                labels4 = batch.Label5.to(device)
                labels5 = batch.Label6.to(device)
                labels6 = batch.Label7.to(device)
                labels7 = batch.Label8.to(device)
                labels8 = batch.Label9.to(device)
                labels9 = batch.Label10.to(device)
                labels10 = batch.Label11.to(device)
                labels11 = batch.Label12.to(device)
                labels12 = batch.Label13.to(device)
                labels13 = batch.Label14.to(device)
                pa = torch.stack([labels0, labels1, labels2, labels3, labels4,labels5,labels6,labels7,labels8,labels9, labels10, labels11,labels12,labels13], dim = 1)
                
                pa = torch.tensor(pa, dtype=torch.float32)
                
                #pa2 = torch.tensor(inputs, dtype=torch.float32)
             # optimizerを初期化
                #optimizer1.zero_grad()
                #optimizer2.zero_grad()

                # 順伝搬（forward）計算
                #if phase == "train":
                    
                Outputs11, BERT_atten11, BERT_cls11 = net_trained11(inputs)
                Outputs12, BERT_atten12, BERT_cls12 = net_trained12(inputs)

                Outputs21, BERT_atten21, BERT_cls21 = net_trained21(inputs)
                Outputs22, BERT_atten22, BERT_cls22 = net_trained22(inputs)
      
                Outputs31, BERT_atten31, BERT_cls31 = net_trained31(inputs)
                Outputs32, BERT_atten32, BERT_cls32 = net_trained32(inputs)
                
                Outputs41, BERT_atten41, BERT_cls41 = net_trained41(inputs)
                Outputs42, BERT_atten42, BERT_cls42 = net_trained42(inputs)
    
                Outputs51, BERT_atten51, BERT_cls51 = net_trained51(inputs)
                Outputs52, BERT_atten52, BERT_cls52 = net_trained52(inputs)
              
                Outputs61, BERT_atten61, BERT_cls61 = net_trained61(inputs)
                Outputs62, BERT_atten62, BERT_cls62 = net_trained62(inputs)
                
                Outputs71, BERT_atten71, BERT_cls71 = net_trained71(inputs)
                Outputs72, BERT_atten72, BERT_cls72 = net_trained72(inputs)

                b1.append(BERT_cls11.cpu().detach().numpy())
                b2.append(BERT_cls21.cpu().detach().numpy())
                b3.append(BERT_cls31.cpu().detach().numpy())
                b4.append(BERT_cls41.cpu().detach().numpy())
                b5.append(BERT_cls51.cpu().detach().numpy())
                b6.append(BERT_cls61.cpu().detach().numpy())
                b7.append(BERT_cls71.cpu().detach().numpy())
                
                c1.append(BERT_cls12.cpu().detach().numpy())
                c2.append(BERT_cls22.cpu().detach().numpy())
                c3.append(BERT_cls32.cpu().detach().numpy())
                c4.append(BERT_cls42.cpu().detach().numpy())
                c5.append(BERT_cls52.cpu().detach().numpy())
                c6.append(BERT_cls62.cpu().detach().numpy())
                c7.append(BERT_cls72.cpu().detach().numpy())
                p.append(pa)
                ins.append(inputs)


                #if phase == "val":



    return b1, b2, b3, b4,b5,b6,b7,c1,c2,c3,c4,c5,c6,c7, p, ins

エポック 1 で CLS トークンをすべて抽出. 30000 データで 1 時間弱かかります.

In [ ]:
num_epochs = 1
b1,b2,b3,b4,b5,b6,b7,c1,c2,c3,c4,c5,c6,c7, p, inp = train_model(bremo_ca, bremo_se, dinmo_ca, dinmo_se, batmo_ca, sermo_se, sermo_ca, stamo_se, stamo_ca, stamo_se, facmo_ca, facmo_se, roomo_ca, roomo_se, dataloaders_dict, criterion, num_epochs=num_epochs)

抽出した CLS トークンを test1 などに格納して Transformer で学習する関数

In [ ]:
def tr_test(net_trained):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


  net_trained.eval()   # モデルを検証モードに
  net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
  epoch_corrects = 0
  pred_list1 = []
  label_list1 = []
  aa1 = 0
  bb1 = 0


  acc_count=0
  part_acc_count=0
  all_miss=0

  # データローダーからミニバッチを取り出すループ
  for (b12, b22, b32, b42,b52,b62,b72,c12,c22,c32,c42,c52,c62,c72, p2) in zip(test1, test2, test3, test4, test5, test6, test7, test8, test9, test10, test11, test12, test13, test14, test_target):
  #zip(b1[vallen],b2[vallen],b3[vallen],b4[vallen],b5[vallen],b6[vallen],b7[vallen],c1[vallen],c2[vallen],c3[vallen],c4[vallen],c5[vallen],c6[vallen],c7[vallen], p[vallen]):
    labels0 = torch.tensor(b12).to(device)  # ラベル
    labels1 = torch.tensor(b22).to(device)
    labels2 = torch.tensor(b32).to(device)
    labels3 = torch.tensor(b42).to(device)
    labels4 = torch.tensor(b52).to(device)
    labels5 = torch.tensor(b62).to(device)
    labels6 = torch.tensor(b72).to(device)
    labels7 = torch.tensor(c12).to(device)
    labels8 = torch.tensor(c22).to(device)
    labels9 = torch.tensor(c32).to(device)
    labels10 = torch.tensor(c42).to(device)
    labels11 = torch.tensor(c52).to(device)
    labels12 = torch.tensor(c62).to(device)
    labels13 = torch.tensor(c72).to(device)
    pa = torch.tensor(p2, dtype=torch.float32).to(device)
                
    pa2 = pa.cpu().numpy()
    # optimizerを初期化
    #label_class1 = pa2[:, :2]
    #0クラス目のデータの諸々を取得し、学習#######################################################
    #pa31 = Pa_Numpy(label_class1)
    #正解ラベル
    #pa4_0 = torch.tensor(pa31, dtype=torch.int64).to(device)
  
    tr_cls = net_trained(labels7, labels0, labels8, labels1, labels9, labels2, labels10, labels3, labels11, labels4, labels12, labels5, labels13, labels6)
  #loss1 = criterion(tr_cls, )

    preds = torch.round(tr_cls)  # ラベルを予測

    aa = preds.cpu().detach().numpy()
    bb = pa.cpu().detach().numpy()
    pred_list1 = np.append(pred_list1, aa)
    label_list1 = np.append(label_list1, bb)

  B=label_list1.reshape(int(len(label_list1)/14), 14)
  A=pred_list1.reshape(int(len(pred_list1)/14), 14)

  PRED_LIST = A
  LABEL_LIST = B

  for i in range(len(PRED_LIST)):
    if (PRED_LIST[i]==LABEL_LIST[i]).all():
      acc_count+=1
    elif (PRED_LIST[i]==LABEL_LIST[i]).any():
      part_acc_count+=1
  all_miss = int(len(test1)*4) - (acc_count + part_acc_count)
  
  print("######################################################")
  print("ACC_COUNT", acc_count)
  print("PART_ACC_COUNT", part_acc_count)
  print("ALL_MISS", all_miss)
  
  return A, B

##################################################################################################################
A,B = tr_test(tr_encoder)

precisionscore = precision_score(A,B, average="micro")
recallscore = recall_score(A,B, average="micro")
f1score = f1_score(A,B, average="micro")
print("テストデータ :", int(len(test1)*4))
print("VAL_Precision", precisionscore)
print("VAL_Recall", recallscore)
print("VAL_F1", f1score)

all_c=[]
part_c=[]

PRED_LIST = A
LABEL_LIST = B
acc_count=0
part_acc_count=0
all_miss=0

for i in range(len(PRED_LIST)):
    if (PRED_LIST[i]==LABEL_LIST[i]).all():
      all_c.append(i)
      acc_count+=1
    elif (PRED_LIST[i]==LABEL_LIST[i]).any():
      part_c.append(i)
      part_acc_count+=1
all_miss = int(len(test1)*4) - (acc_count + part_acc_count)
  
print("######################################################")
print("ACC_COUNT", acc_count)
print("PART_ACC_COUNT", part_acc_count)
print("ALL_MISS", all_miss)